<a href="https://colab.research.google.com/github/JahagirdarPrajwal/Sentiment-Analysis-using-NLP/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

**Uplaoding kaggle file **

In [ ]:
# configure the path
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


**IMPORTING TWITTER SENTIMENT **

In [ ]:
# api to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 74% 60.0M/80.9M [00:00<00:00, 233MB/s]
100% 80.9M/80.9M [00:00<00:00, 200MB/s]


In [ ]:
# extracting the compressed dataset

from zipfile import ZipFile # reads zipfile
zdataset = '/content/sentiment140.zip'

with ZipFile(zdataset,'r') as zip:
  zip.extractall()
  print('Done')


Done


**IMPORTING THE DEPENDENCIES**

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer # used to feed the data from vector to numerical
from sklearn.model_selection import train_test_split # used to split the data into training and test
from sklearn.linear_model import LogisticRegression # training data
from sklearn.metrics import accuracy_score #accuracy calculator

In [ ]:
import nltk
nltk.download('stopwords') # stopwords are those which doesnt have influential meaning to the data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# printing the stopwords in english
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#laoding the data from csv file
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

**Data Processing **

In [ ]:
# chechking the number of rows and column
twitter_data.shape # 16lakhs reach as in python it starts from 0 ( 16 laksh rows and 6 column)

(1599999, 6)

In [ ]:
# printing the first 5 rows
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns and reading the dataset again as its reading the heading of the table as a individual column

column_names = ['target','id','date','flag','user','text'] # data given on the websitez
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names,encoding='ISO-8859-1')

In [ ]:
# chechking the number of rows and column
twitter_data.shape # the first tweet was read as column name

(1600000, 6)

In [ ]:
# printing the first 5 rows
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# chechking and counting of missing values
twitter_data.isnull().sum() # isnull helps for missing values

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#CHECHKING   THE DISTRIBUTION OF TARGET COLUMN
twitter_data['target'].value_counts() # data if 0 means negative and data if 4 means positive data has equal distribution so its working otherwise would have to make it equal to make it work

target
0    800000
4    800000
Name: count, dtype: int64

**CONVERTING TARGET FROM 4 TO 1 **

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
#CHECHKING   THE DISTRIBUTION OF TARGET COLUMN
twitter_data['target'].value_counts() # converted it from 4 to 1

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative Tweet


1 --> Positive Tweet

**STEAMING**

process of reducing a word to its root word like actor , actress , acting = act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):

  steamed_content = re.sub ('[^a-zA-Z]',' ',content) # removing everything which is not letter
  steamed_content = steamed_content.lower() # converting uppercase to lowercase
  steamed_content = steamed_content.split() # spliting the words
  steamed_content = [port_stem.stem(word) for word in steamed_content if not word in stopwords.words('english')] # check for stopwords
  steamed_content = ' '.join(steamed_content)

  return steamed_content

In [ ]:
twitter_data['stemmed_contet'] = twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_contet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(twitter_data['stemmed_contet'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_contet, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target']) # we dont need id data flag for checking if its negative or positive

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# separting the data and label
X = twitter_data['stemmed_contet'].values # all the tweets are stored here
Y = twitter_data['target'].values # all the target values are stored here

In [ ]:
print(X) # printing variables seperately

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y) # printing variables seperately

[0 0 0 ... 1 1 1]


**SPLITNG THE DATA TO TRAIN AND TEST**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2) # X_train contains all the training data and labels in y_train y test contains all the negative and positive for all the xtest
# spliting the x and y test size 0.2 = 20 percent data will go for test purpose and rest for training
#  stratify will give the almost equal proportion of training data and random state 2 will replicate

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [ ]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
# converting the text data to numerical data

vectorizer = TfidfVectorizer() # used for convert text to numerical for model to unerstand
# will check the no of times a word has occured and get the importance required
# if a word was occured many times in a positive term then it will be termed as positive

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train) # starting 0 means words in the first tweet and similarly 1 means tweet in the second tweet

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [ ]:
print(X_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

**TRAINING THE MACHINE LEARNING MODEL **

logistic regression
  -classification model



In [ ]:
model = LogisticRegression(max_iter=1000) # max no of times model will go through the data

In [ ]:
model.fit(X_train, Y_train) # x train is training data y train is target for training data
# feeding the xtrain and y train to the model

LogisticRegression(max_iter=1000)

**Model Evaluation**

Accuracy Score

In [ ]:
# Accuracy score of the train data
X_train_prediction = model.predict(X_train) # predits for 0 and 1
training_data_accuracy = accuracy_score(Y_train, X_train_prediction) # accuracy score is the function

In [ ]:
print('Accuracy score of the data : ', training_data_accuracy)

Accuracy score of the data :  0.81018984375


In [ ]:
# Accuracy score of the test data
X_test_prediction = model.predict(X_test) # predits for 0 and 1
test_data_accuracy = accuracy_score(Y_test, X_test_prediction) # accuracy score is the function

In [ ]:
print('Accuracy score of the data : ', test_data_accuracy) # as the accuracy percentage for both test and train data are close we can say the model worked properly

Accuracy score of the data :  0.7780375


**Model Accuracy = 77.8%**

**Saving the trained model**


In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb')) # dumping all the saved trained model

**Using the saved model for future prediction**

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb')) # rb is reading the file in binary format

In [ ]:
X_new = X_test[350]
print(Y_test[350])

prediction = model.predict(X_new) # saves the new prediction
print(prediction) # prints the new value is 0 or 1 \

if prediction[0] == 0:
  print('The tweet is negative')
else:
  print('The tweet is positive')

  # [1] is a list

0
[0]
The tweet is negative


In [ ]:
X_new = X_test[100]
print(Y_test[100])

prediction = model.predict(X_new) # saves the new prediction
print(prediction) # prints the new value is 0 or 1 \

if prediction[0] == 0:
  print('The tweet is negative')
else:
  print('The tweet is positive')

  # [1] is a list

0
[0]
The tweet is negative
